<a href="https://colab.research.google.com/github/Kim-TaeKyoung/a/blob/main/0_3_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python

In [ ]:
# -*- coding: utf-8 -*-
import json
from glob import glob
import cv2
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_path = glob("/content/drive/MyDrive/Colab/AI_ChampionShip/dataset/raw/kdx-dataset/train/*.mp4")
test_path = glob("/content/drive/MyDrive/Colab/AI_ChampionShip/dataset/raw/kdx-dataset/test/*.mp4")
annJson = glob("/content/drive/MyDrive/Colab/AI_ChampionShip/dataset/raw/kdx-dataset/train/*.json")
TrainList = sorted(train_path)

with open(annJson[0], 'r') as f:
    fire_scenes = json.load(f)
fireScenesList = list(fire_scenes)

num_video_files = len(fire_scenes.keys())
print('Train 영상은 {}개 입니다'.format(num_video_files))
print('Test 영상은 {}개 입니다'.format(len(test_path)))
fgbg = cv2.createBackgroundSubtractorMOG2()
kernel = np.ones((5, 5), np.uint8)

for idx, val in enumerate(fireScenesList):
    vidCap = cv2.VideoCapture(TrainList[idx])
    jsonParse = fire_scenes[val]

    if not jsonParse['scenes']:
        continue

    else:

        if not vidCap.isOpened():
            print("could not open :", val)
            exit(0)

        if val == TrainList[idx].split('/')[-1]:
            width = vidCap.get(cv2.CAP_PROP_FRAME_WIDTH)
            height = vidCap.get(cv2.CAP_PROP_FRAME_HEIGHT)
            fps = round(vidCap.get(cv2.CAP_PROP_FPS))
            print('Video Name: ', val)
            print('Video Resolution: %d, %d' % (width, height))
            print('Frame Count:', int(vidCap.get(cv2.CAP_PROP_FRAME_COUNT)))
            print('openCV_FPS:', fps)
            print('PlayTime:', jsonParse['duration'])
            print('FPS:', vidCap.get(cv2.CAP_PROP_FRAME_COUNT) / jsonParse['duration'])
            print(vidCap.get(cv2.CAP_PROP_POS_AVI_RATIO))
            print(vidCap.get(cv2.CAP_PROP_POS_MSEC))

            n = 0
            k = len(jsonParse['scenes'])
            videoCount = 1
            print(jsonParse['scenes'])
            print('Label 개수:', k)
            print('==' * 10)

            while vidCap.isOpened():
                ret, image = vidCap.read()

                if not ret:
                    break

                # 이미지 리사이즈
                frame = cv2.resize(image, (1000, 600))
                #가우시안 블러처리
                blur = cv2.GaussianBlur(frame, (21, 21), 0)
                #HSV로 변환
                hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
                #HSV 경계값 설정
                lower = [18, 50, 50]
                upper = [35, 255, 255]

                lower = np.array(lower, dtype='uint8')
                upper = np.array(upper, dtype='uint8')
                #이미지에 HSV 경계의 마스크 설정
                mask = cv2.inRange(hsv, lower, upper)
                #HSV 경계의 마스크 기준으로 이미지 표현
                output = cv2.bitwise_and(frame, hsv, mask=mask)
                number_of_total = cv2.countNonZero(mask)

                cv2.imshow("original", frame)
                cv2.imshow("Fire", output)

                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
    vidCap.release()
    cv2.destroyAllWindows()